In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [2]:
filename = '2022_India_persona_groups_cleaned'

In [3]:
import pandas as pd

df = pd.read_csv(f"data/{filename}.csv")

rows, cols = df.shape
print(f"Rows: {rows}, Columns: {cols}")

Rows: 74, Columns: 11


In [ ]:
import json
with open("data/questions.json", "r") as f:
    questions = json.load(f)
with open("data/chosen_cols_gemma.json", "r") as f:
    chosen_cols = json.load(f)

chosen_qsns = {}

for qsn in questions:
    if chosen_cols['chosen_cols'][qsn] == True and questions[qsn]['description'] not in chosen_cols['persona_cols']:
        chosen_qsns[qsn] = questions[qsn]

print("Number of questions in the questionnaire - ", len(chosen_qsns))

Number of questions in the questionnaire -  80


## Chosen Persona Features

In [5]:
region = 'N_REGION_ISO: Region ISO 3166-2'
urban_rural = 'H_URBRURAL: Urban-Rural'
age = 'X003R: Age recoded (6 intervals)'
gender = 'Q260: Sex'
language = 'Q272: Language at home'
marital_status = 'Q273: Marital status'
education_level = 'Q275R: Highest educational level: Respondent (recoded into 3 groups)'
social_class = 'Q287: Social class (subjective)'

In [6]:
general_prompt_en = '''
Imagine you are a {language}-speaking {marital_status} {gender} from {urban_rural} {region}, India. You are in {age} years of age category and have completed {education_level} education level. You consider yourself part of the {social_class}. Answer the following question from this perspective. Others will read what you choose; your goal is to convince them it was chosen from the perspective of the persona described above.

Select exactly one option. Answer ONLY with the number corresponding to the question, followed by the number corresponding to the chosen option. Do NOT repeat the question or any other text.
'''

user_prompt_en = '''
Q: {Question}
Options: {Options}
A:
'''


In [7]:
df.iloc[0]

A_YEAR: Year of survey                                                                2023
B_COUNTRY: ISO 3166-1 numeric country code                                           India
N_REGION_ISO: Region ISO 3166-2                                                IN-BR Bihar
H_URBRURAL: Urban-Rural                                                              Rural
Q260: Sex                                                                           Female
X003R: Age recoded (6 intervals)                                                     16-24
Q272: Language at home                                                               Hindi
Q273: Marital status                                                                Single
Q275R: Highest educational level: Respondent (recoded into 3 groups)                Higher
Q287: Social class (subjective)                                         Lower middle class
Counts                                                                                   4

## Prompting gemma

In [8]:
df = df.iloc[0:5]

In [9]:
import pandas as pd
import re
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time 

In [10]:
# --- 1. Load the Local Model ---
# This section replaces the initial Gemini API setup.
# Make sure to update the path to where your model is stored.
model_path = "/assets/models/google-gemma-3-it-27b" 

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Loading model... This might take a moment.")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # Automatically uses your GPUs
    torch_dtype=torch.bfloat16 # Efficient data type for modern GPUs
)
print("Model loaded successfully.")



Loading tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading model... This might take a moment.


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

Model loaded successfully.


In [11]:

batch_size = 10
results = []
raw_results = []
respondent_number = 0

for _, row in df.iterrows():
    respondent_number += 1
    general_context = {
        "language": row[language],
        "marital_status": row[marital_status],
        "gender": row[gender],
        "urban_rural": row[urban_rural],
        "region": row[region],
        "age": row[age],
        "education_level": row[education_level],
        "social_class": row[social_class]
    }

    questions = []
    for qsn_key in chosen_qsns:
        qsn_text = chosen_qsns[qsn_key]['questions'][0]
        options_list = chosen_qsns[qsn_key]['options']
        options_text = "".join([f"{idx+1}. {opt} " for idx, opt in enumerate(options_list)])
        questions.append((qsn_key, qsn_text, options_list, options_text))

    respondent_answers = general_context.copy()
    debug_output = {"persona": general_context, "questions": []}

    for i in tqdm(range(0, len(questions), batch_size), desc=f"Processing question batches for respondent {respondent_number}"):
        batch = questions[i:i+batch_size]

        user_prompt = ""
        for idx, (_, q_text, _, opts_text) in enumerate(batch, start=1):
            user_prompt += f"Question {idx}: {q_text}\nOptions: {opts_text}\n"
        user_prompt += "\nAnswer ONLY with numbers in format: Q1: <option_number>, Q2: <option_number>, ... Do NOT repeat questions."

        messages = [
            {"role": "system", "content": general_prompt_en.format(**general_context)},
            {"role": "user", "content": user_prompt}
        ]

        if hasattr(tokenizer, 'apply_chat_template'):
            formatted_prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )
        else:
            system_content = general_prompt_en.format(**general_context)
            formatted_prompt = f"<s>[INST] <<SYS>>\n{system_content}\n<</SYS>>\n\n{user_prompt} [/INST]"

        inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True).to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.0,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id, 
            do_sample=False 
        )

        generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        raw_results.append({
            "question_batch": user_prompt,
            "formatted_prompt": formatted_prompt,
            "answer_text": answer_text
        })

        batch_answers = re.findall(r'Q\d+:\s*(\d+)', answer_text)

        for j, (qsn_key, q_text, opts_list, _) in enumerate(batch):
            if j < len(batch_answers):
                ans_idx = int(batch_answers[j]) - 1
                if 0 <= ans_idx < len(opts_list):
                    ans_value = opts_list[ans_idx]
                else:
                    ans_value = "Invalid answer"
            else:
                ans_value = "No answer"
                
            respondent_answers[qsn_key] = ans_value
            debug_output["questions"].append({
                "question_key": qsn_key,
                "question_text": q_text,
                "options": opts_list,
                "answer_id": batch_answers[j] if j < len(batch_answers) else None,
                "answer_value": ans_value
            })

    results.append(respondent_answers)
    if respondent_number % 10 == 0:
        results_df = pd.DataFrame(results)
        results_df.to_csv("survey_answers_wide.csv", index=False)


Processing question batches for respondent 1:   0%|          | 0/8 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Processing question batches for respondent 5: 100%|██████████| 8/8 [02:31<00:00, 18.97s/it]


In [12]:
results_df = pd.DataFrame(results)
results_df.to_csv("survey_gemma_answers_wide.csv", index=False)

with open("temp/survey_gemma_answers_debug.json", "w", encoding="utf-8") as f:
    json.dump(raw_results, f, indent=4, ensure_ascii=False)

print("Processing complete.")

FileNotFoundError: [Errno 2] No such file or directory: 'temp/survey_gemma_answers_debug.json'